Librerías

In [1]:
import pandas as pd 
from sqlalchemy import create_engine
import os
from math import ceil
%matplotlib inline 
from functools import reduce
pd.set_option('display.max_columns',500)

Conexión

In [2]:
df = pd.read_csv('/home/lu/Documents/Mineria /Datos_red/nyc-rolling-sales.csv',encoding='utf8')

In [3]:
df.columns = [x.lower() for x in df.columns]

In [4]:
df.head()

,unnamed: 0,borough,neighborhood,building class category,tax class at present,block,lot,ease-ment,building class at present,address,apartment number,zip code,residential units,commercial units,total units,land square feet,gross square feet,year built,tax class at time of sale,building class at time of sale,sale price,sale date
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


ETL

In [5]:
df['class_category'] = df['building class category'].map(lambda x:int(x[:2]))
#df['class_category'] 

In [6]:
df['limpia_fh']=df['sale date'].map(lambda x:x.replace(' ','|').split('|'))
df[['fecha','hora']]=df.limpia_fh.apply(pd.Series)

In [7]:
df['fecha']=pd.to_datetime(df.fecha)

df[u'año']=df['fecha'].map(lambda x:x.year)
df[u'mes']=df['fecha'].map(lambda x:x.month)
df[u'dia']=df['fecha'].map(lambda x:x.day)

Dimensiones

d_colonia

In [8]:
lst_colonia = ['neighborhood']
tb_colonia = df [lst_colonia].copy()
tb_colonia = tb_colonia.drop_duplicates()
tb_colonia.reset_index(drop=True,inplace=True)
tb_colonia['id_colonia']=tb_colonia.index+1
tb_colonia.head()

,neighborhood,id_colonia
0,ALPHABET CITY,1
1,CHELSEA,2
2,CHINATOWN,3
3,CIVIC CENTER,4
4,CLINTON,5


d_ciudad

In [9]:
lst_ciudad= ['borough']
tb_ciudad = df [lst_ciudad].copy()
##tb_ciudad ['name_b'] = ['Manhattan','Bronx','Brooklyn','Queens','Staten Island']
tb_ciudad = tb_ciudad.drop_duplicates()
tb_ciudad.reset_index(drop=True,inplace=True)
tb_ciudad['id_ciudad']=tb_ciudad.index+1
tb_ciudad.head()

,borough,id_ciudad
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [10]:
tb_ciudad.head()

,borough,id_ciudad
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


d_zip

In [11]:
lst_zip = ['zip code']
tb_zip = df [lst_zip].copy()
tb_zip = tb_zip.drop_duplicates()



In [12]:
tb_zip.reset_index(drop=True,inplace=True)
tb_zip['id_zip']=tb_zip.index+1
tb_zip.head()

,zip code,id_zip
0,10009,1
1,0,2
2,10002,3
3,10011,4
4,10001,5


d_tax

In [13]:
lst_taxp = ['tax class at present']
tb_taxp = df [lst_taxp].copy()
tb_taxp = tb_taxp.drop_duplicates()


In [14]:
tb_taxp.reset_index(drop=True,inplace=True)
tb_taxp['id_taxp']=tb_taxp.index+1
tb_taxp.head()

,tax class at present,id_taxp
0,2A,1
1,2,2
2,2B,3
3,2C,4
4,,5


d_class_categorry

In [15]:
lst_cc = ['class_category']
tb_cc = df [lst_cc].copy()
tb_cc = tb_cc.drop_duplicates()

In [16]:
tb_cc.reset_index(drop=True,inplace=True)
tb_cc['id_cc']=tb_cc.index+1
tb_cc.head()

,class_category,id_cc
0,7,1
1,8,2
2,9,3
3,10,4
4,11,5


d_class_present

In [17]:
lst_cp = ['building class at present']
tb_cp = df [lst_cp].copy()
tb_cp = tb_cp.drop_duplicates()

In [18]:
tb_cp.reset_index(drop=True,inplace=True)
tb_cp['id_cp']=tb_cp.index+1
tb_cp.head()

,building class at present,id_cp
0,C2,1
1,C7,2
2,C4,3
3,D5,4
4,D9,5


d_year_b

In [19]:
lst_yb = ['year built']
tb_yb = df [lst_yb].copy()
tb_yb = tb_yb.drop_duplicates()

In [20]:
tb_yb.reset_index(drop=True,inplace=True)
tb_yb['id_yb']=tb_yb.index+1
tb_yb.head()

,year built,id_yb
0,1900,1
1,1913,2
2,1920,3
3,1910,4
4,2009,5


d_tax_sale

In [21]:

lst_taxs = ['tax class at time of sale']
tb_taxs = df [lst_taxs].copy()
tb_taxs = tb_taxs.drop_duplicates()

In [22]:
tb_taxs.reset_index(drop=True,inplace=True)
tb_taxs['id_taxs']=tb_taxs.index+1
tb_taxs.head()

,tax class at time of sale,id_taxs
0,2,1
1,4,2
2,1,3
3,3,4


d_tiempo

In [23]:
lst_salestime = ['año','mes','dia']
tb_salestime=df[lst_salestime].copy()
#tb_salestime
tb_salestime.shape

(84548, 3)

In [24]:
tb_salestime = tb_salestime.drop_duplicates()
#tb_salestime
tb_salestime.shape

(364, 3)

In [25]:
##tb_salestime.sort_values(['año', 'mes'],ascending=[1,1],inplace=True)
tb_salestime.reset_index(drop=True,inplace=True)
tb_salestime['id_salestime']=tb_salestime.index+1
tb_salestime.head()

,año,mes,dia,id_salestime
0,2017,7,19,1
1,2016,12,14,2
2,2016,12,9,3
3,2016,9,23,4
4,2016,11,17,5


d_block

In [26]:
lst_b = ['block']
tb_b = df [lst_b].copy()
tb_b = tb_b.drop_duplicates()
tb_b.reset_index(drop=True,inplace=True)
tb_b['id_b']=tb_b.index+1
tb_b.head()

,block,id_b
0,392,1
1,399,2
2,402,3
3,404,4
4,405,5


In [27]:
lst_t = ['lot']
tb_t = df [lst_t].copy()
tb_t = tb_t.drop_duplicates()
tb_t.reset_index(drop=True,inplace=True)
tb_t['id_t']=tb_t.index+1
tb_t.head()

,lot,id_t
0,6,1
1,26,2
2,39,3
3,21,4
4,55,5


mando de id's a la tabla principal

In [28]:
df=df.merge(tb_cc,on=lst_cc,how='inner')
df.drop(lst_cc,axis=1,inplace=True)

In [29]:
df=df.merge(tb_ciudad,on=lst_ciudad,how='inner')
df.drop(lst_ciudad,axis=1,inplace=True)

In [30]:
df=df.merge(tb_colonia,on=lst_colonia,how='inner')
df.drop(lst_colonia,axis=1,inplace=True)

In [31]:
df=df.merge(tb_taxp,on=lst_taxp,how='inner')
df.drop(lst_taxp,axis=1,inplace=True)

In [32]:
df=df.merge(tb_taxs,on=lst_taxs,how='inner')
df.drop(lst_taxs,axis=1,inplace=True)

In [33]:
df=df.merge(tb_cp,on=lst_cp,how='inner')
df.drop(lst_cp,axis=1,inplace=True)

In [34]:
df=df.merge(tb_zip,on=lst_zip,how='inner')
df.drop(lst_zip,axis=1,inplace=True)

In [35]:
df=df.merge(tb_salestime,on=lst_salestime,how='inner')
df.drop(lst_salestime,axis=1,inplace=True)

In [36]:
df=df.merge(tb_yb,on=lst_yb,how='inner')
df.drop(lst_yb,axis=1,inplace=True)

In [37]:
df=df.merge(tb_b,on=lst_b,how='inner')
df.drop(lst_b,axis=1,inplace=True)

In [38]:
df=df.merge(tb_t,on=lst_t,how='inner')
df.drop(lst_t,axis=1,inplace=True)

In [39]:
df.shape

(84548, 27)

In [40]:
df.head()

,unnamed: 0,building class category,ease-ment,address,apartment number,residential units,commercial units,total units,land square feet,gross square feet,building class at time of sale,sale price,sale date,limpia_fh,fecha,hora,id_cc,id_ciudad,id_colonia,id_taxp,id_taxs,id_cp,id_zip,id_salestime,id_yb,id_b,id_t
0,4,07 RENTALS - WALKUP APARTMENTS,,153 AVENUE B,,5,0,5,1633,6440,C2,6625000,2017-07-19 00:00:00,"[2017-07-19, 00:00:00]",2017-07-19,00:00:00,1,1,1,1,1,1,1,1,1,1,1
1,1868,10 COOPS - ELEVATOR APARTMENTS,,"170 SECOND AVENUE, 12D",,0,0,0,-,-,D4,1800000,2017-06-09 00:00:00,"[2017-06-09, 00:00:00]",2017-06-09,00:00:00,4,1,6,2,1,10,14,13,8,175,1
2,1865,10 COOPS - ELEVATOR APARTMENTS,,"170 2ND AVENUE, 8B",,0,0,0,-,-,D4,2650000,2017-01-17 00:00:00,"[2017-01-17, 00:00:00]",2017-01-17,00:00:00,4,1,6,2,1,10,14,37,8,175,1
3,1863,10 COOPS - ELEVATOR APARTMENTS,,"170 SECOND AVENUE, 12C",,0,0,0,-,-,D4,940000,2016-09-29 00:00:00,"[2016-09-29, 00:00:00]",2016-09-29,00:00:00,4,1,6,2,1,10,14,54,8,175,1
4,1862,10 COOPS - ELEVATOR APARTMENTS,,"170 2ND AVE, 12B",,0,0,0,-,-,D4,3025000,2016-09-13 00:00:00,"[2016-09-13, 00:00:00]",2016-09-13,00:00:00,4,1,6,2,1,10,14,206,8,175,1


creando tabla hechos

In [48]:
df = df[['sale price','residential units','commercial units','total units',#'gross square feet',
         'id_cc',
        'id_ciudad','id_colonia','id_taxp','id_taxs','id_cp','id_zip','id_salestime','id_yb','id_b','id_t']]

In [42]:
#df['n']=1

In [49]:
aux=df.pivot_table(index=['id_cc','id_ciudad','id_colonia','id_taxp','id_taxs','id_cp','id_zip',
                          'id_salestime','id_yb','id_b','id_t'],)#,aggfunc='sum',values=['sale price','n'],fill_value=0)
                   

In [51]:
aux.reset_index(inplace=True)


In [52]:
aux.head()

,id_cc,id_ciudad,id_colonia,id_taxp,id_taxs,id_cp,id_zip,id_salestime,id_yb,id_b,id_t,commercial units,residential units,total units
0,1,1,1,1,1,1,1,1,1,1,1,0.0,5.0,5.0
1,1,1,1,1,1,1,1,5,1,4,5,0.0,6.0,6.0
2,1,1,1,2,1,2,1,2,1,2,2,3.0,28.0,31.0
3,1,1,1,2,1,2,1,3,1,2,3,1.0,16.0,17.0
4,1,1,1,2,1,2,1,6,1,7,8,2.0,44.0,46.0
